# Breast Cancer Histology Image Classification Using VGG16 (Transfer Learning)
***
## Description
This code implements a binary classification model to distinguish between benign and malignant breast cancer histology images using transfer learning with the VGG16 architecture.

## Key Steps

### 1. Data Loading & Preparation
- Loads images from two directories:
  - `benign_dir`: Contains benign tumor histology slides (class 0)
  - `malignant_dir`: Contains malignant tumor histology slides (class 1)
- Uses `train_test_split()` to create a 70-30 train-test split (stratified to maintain class balance).

### 2. Image Preprocessing
- Images are:
  - Resized to 224x224 (VGG16 input size)
  - Normalized to [0, 1]
  - Converted into a TensorFlow Dataset for efficient batching.

### 3. Model Architecture (Transfer Learning)
- Uses VGG16 (pre-trained on ImageNet) as a feature extractor.
- Freezes all VGG16 layers to retain pre-trained weights.
- Adds a custom classification head:
  - `Flatten()` → `Dense(128, ReLU)` → `Dense(1, Sigmoid)` (binary output).

### 4. Training & Evaluation
- Compiles the model with:
  - Optimizer: Adam
  - Loss: Binary Crossentropy
  - Metric: Accuracy
- Trains for 10 epochs on the training set.
- Evaluates on the test set and computes predictions (threshold > 0.5 for class assignment).

## Key Metrics
- **F1-score** and **Geometric Mean Score (G-mean)** are imported (though not explicitly computed in this snippet).
- Model performance can be further analyzed using confusion matrices, ROC curves, or precision-recall metrics.

## Usage Notes
- Ensure the dataset path (`/content/drive/MyDrive/Datasets/...`) is correctly mounted in Google Colab.
- Adjust `BATCH_SIZE` and `EPOCHS` based on available computational resources.
- For imbalanced datasets, consider class weights or augmentation techniques to improve generalization.


***
**Magnification Factor: 40X**
***

In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from imblearn.metrics import geometric_mean_score

# Define dataset paths
benign_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/benign/40X"
malignant_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/malignant/40X"

# Function to load image paths
def load_image_paths(dir_path):
    return [os.path.join(dir_path, img) for img in os.listdir(dir_path) if img.endswith('.png')]

benign_images = load_image_paths(benign_dir)
malignant_images = load_image_paths(malignant_dir)

print(f"Total Benign Images: {len(benign_images)}")
print(f"Total Malignant Images: {len(malignant_images)}")

# Create labels (0 = Benign, 1 = Malignant)
benign_labels = [0] * len(benign_images)
malignant_labels = [1] * len(malignant_images)

# Combine images and labels
all_images = np.array(benign_images + malignant_images)
all_labels = np.array(benign_labels + malignant_labels)

# Split into training (70%) and testing (30%)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.3, stratify=all_labels, random_state=42
)

print(f"Training samples: {len(train_images)}")
print(f"Testing samples: {len(test_images)}")

# Function to preprocess images
def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)  # Decode PNG images
    img = tf.image.resize(img, [224, 224])  # Resize to 224x224
    img = img / 255.0  # Normalize pixel values
    return img, label

# Create TensorFlow datasets
BATCH_SIZE = 128

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.map(process_path).shuffle(1000).batch(BATCH_SIZE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = test_dataset.map(process_path).batch(BATCH_SIZE)

# Ensure testing dataset is not empty
if sum(1 for _ in test_dataset) == 0:
    raise ValueError("Testing dataset is empty. Adjust your dataset split.")

# Load VGG16 without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classifier on top
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)  # Binary classification

# Create the model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
EPOCHS = 100
history = model.fit(train_dataset, epochs=EPOCHS)

# Evaluate model
test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()
from sklearn.metrics import confusion_matrix

# Evaluate model
test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()

# Get confusion matrix values
tn, fp, fn, tp = confusion_matrix(test_labels, test_preds).ravel()

# Calculate IBA
iba = (tp / (tp + fn)) + (tn / (tn + fp)) - 1

# Output results
f1 = f1_score(test_labels, test_preds)
gmean = geometric_mean_score(test_labels, test_preds)

loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"G-Mean: {gmean:.4f}")
print(f"Informedness (IBA): {iba:.4f}")

Total Benign Images: 626
Total Malignant Images: 1370
Training samples: 1397
Testing samples: 599
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 148s 4s/step - accuracy: 0.5984 - loss: 1.6343
Epoch 2/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 21s 720ms/step - accuracy: 0.7069 - loss: 0.5416
Epoch 3/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 41s 822ms/step - accuracy: 0.8365 - loss: 0.3649
Epoch 4/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 19s 765ms/step - accuracy: 0.9150 - loss: 0.2735
Epoch 5/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 21s 743ms/step - accuracy: 0.9131 - loss: 0.2511
Epoch 6/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 19s 824ms/step - accuracy: 0.9312 - loss: 0.2187
Epoch 7/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 20s 719ms/step - accuracy: 0.9502 - loss: 0.1880
Epoch 8/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 20s 766ms/step - accuracy: 0.9517 - loss: 0.1666
Epoch 9/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 19s 722ms/step - accuracy: 0.9545 - loss: 0.1557
Epoch 10/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 20s 872ms/step - accu

***
**Magnification Factor: 100X**
***

In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from imblearn.metrics import geometric_mean_score

# Define dataset paths
benign_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/benign/100X"
malignant_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/malignant/100X"

# Function to load image paths
def load_image_paths(dir_path):
    return [os.path.join(dir_path, img) for img in os.listdir(dir_path) if img.endswith('.png')]

benign_images = load_image_paths(benign_dir)
malignant_images = load_image_paths(malignant_dir)

print(f"Total Benign Images: {len(benign_images)}")
print(f"Total Malignant Images: {len(malignant_images)}")

# Create labels (0 = Benign, 1 = Malignant)
benign_labels = [0] * len(benign_images)
malignant_labels = [1] * len(malignant_images)

# Combine images and labels
all_images = np.array(benign_images + malignant_images)
all_labels = np.array(benign_labels + malignant_labels)

# Split into training (70%) and testing (30%)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.3, stratify=all_labels, random_state=42
)

print(f"Training samples: {len(train_images)}")
print(f"Testing samples: {len(test_images)}")

# Function to preprocess images
def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)  # Decode PNG images
    img = tf.image.resize(img, [224, 224])  # Resize to 224x224
    img = img / 255.0  # Normalize pixel values
    return img, label

# Create TensorFlow datasets
BATCH_SIZE = 128

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.map(process_path).shuffle(1000).batch(BATCH_SIZE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = test_dataset.map(process_path).batch(BATCH_SIZE)

# Ensure testing dataset is not empty
if sum(1 for _ in test_dataset) == 0:
    raise ValueError("Testing dataset is empty. Adjust your dataset split.")

# Load VGG16 without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classifier on top
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)  # Binary classification

# Create the model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
EPOCHS = 100
history = model.fit(train_dataset, epochs=EPOCHS)

# Evaluate model
test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()
from sklearn.metrics import confusion_matrix

# Evaluate model
test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()

# Get confusion matrix values
tn, fp, fn, tp = confusion_matrix(test_labels, test_preds).ravel()

# Calculate IBA
iba = (tp / (tp + fn)) + (tn / (tn + fp)) - 1

# Output results
f1 = f1_score(test_labels, test_preds)
gmean = geometric_mean_score(test_labels, test_preds)

loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"G-Mean: {gmean:.4f}")
print(f"Informedness (IBA): {iba:.4f}")

Total Benign Images: 649
Total Malignant Images: 1437
Training samples: 1460
Testing samples: 626
Epoch 1/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 113s 4s/step - accuracy: 0.5796 - loss: 1.8177
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 51s 684ms/step - accuracy: 0.7782 - loss: 0.5041
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 20s 668ms/step - accuracy: 0.8110 - loss: 0.3950
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 21s 695ms/step - accuracy: 0.8732 - loss: 0.2958
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 20s 662ms/step - accuracy: 0.9055 - loss: 0.2407
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 21s 695ms/step - accuracy: 0.9392 - loss: 0.2094
Epoch 7/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 19s 648ms/step - accuracy: 0.9401 - loss: 0.1883
Epoch 8/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 20s 668ms/step - accuracy: 0.9573 - loss: 0.1627
Epoch 9/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 21s 823ms/step - accuracy: 0.9716 - loss: 0.1345
Epoch 10/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 19s 658ms/step - accuracy: 0.9776 - loss: 0.1119
Epoch 11/100
12/12 ━━━━

***
**Magnification Factor: 200X**
***

In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from imblearn.metrics import geometric_mean_score

# Define dataset paths
benign_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/benign/200X"
malignant_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/malignant/200X"

# Function to load image paths
def load_image_paths(dir_path):
    return [os.path.join(dir_path, img) for img in os.listdir(dir_path) if img.endswith('.png')]

benign_images = load_image_paths(benign_dir)
malignant_images = load_image_paths(malignant_dir)

print(f"Total Benign Images: {len(benign_images)}")
print(f"Total Malignant Images: {len(malignant_images)}")

# Create labels (0 = Benign, 1 = Malignant)
benign_labels = [0] * len(benign_images)
malignant_labels = [1] * len(malignant_images)

# Combine images and labels
all_images = np.array(benign_images + malignant_images)
all_labels = np.array(benign_labels + malignant_labels)

# Split into training (70%) and testing (30%)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.3, stratify=all_labels, random_state=42
)

print(f"Training samples: {len(train_images)}")
print(f"Testing samples: {len(test_images)}")

# Function to preprocess images
def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)  # Decode PNG images
    img = tf.image.resize(img, [224, 224])  # Resize to 224x224
    img = img / 255.0  # Normalize pixel values
    return img, label

# Create TensorFlow datasets
BATCH_SIZE = 128

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.map(process_path).shuffle(1000).batch(BATCH_SIZE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = test_dataset.map(process_path).batch(BATCH_SIZE)

# Ensure testing dataset is not empty
if sum(1 for _ in test_dataset) == 0:
    raise ValueError("Testing dataset is empty. Adjust your dataset split.")

# Load VGG16 without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classifier on top
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)  # Binary classification

# Create the model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
EPOCHS = 100
history = model.fit(train_dataset, epochs=EPOCHS)

# Evaluate model
test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()
from sklearn.metrics import confusion_matrix

# Evaluate model
test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()

# Get confusion matrix values
tn, fp, fn, tp = confusion_matrix(test_labels, test_preds).ravel()

# Calculate IBA
iba = (tp / (tp + fn)) + (tn / (tn + fp)) - 1

# Output results
f1 = f1_score(test_labels, test_preds)
gmean = geometric_mean_score(test_labels, test_preds)

loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"G-Mean: {gmean:.4f}")
print(f"Informedness (IBA): {iba:.4f}")

Total Benign Images: 623
Total Malignant Images: 1390
Training samples: 1409
Testing samples: 604
Epoch 1/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 157s 3s/step - accuracy: 0.6199 - loss: 1.7467
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 67s 607ms/step - accuracy: 0.4452 - loss: 0.9046
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 21s 726ms/step - accuracy: 0.7088 - loss: 0.5611
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 20s 587ms/step - accuracy: 0.7593 - loss: 0.4746
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 19s 632ms/step - accuracy: 0.7963 - loss: 0.4003
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 18s 589ms/step - accuracy: 0.8157 - loss: 0.3678
Epoch 7/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 21s 608ms/step - accuracy: 0.8644 - loss: 0.3245
Epoch 8/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 20s 596ms/step - accuracy: 0.8596 - loss: 0.3179
Epoch 9/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 21s 609ms/step - accuracy: 0.8565 - loss: 0.3206
Epoch 10/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 22s 588ms/step - accuracy: 0.8807 - loss: 0.2771
Epoch 11/100
12/12 ━━━━

***
**Magnification Factor: 400X**
***

In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from imblearn.metrics import geometric_mean_score

# Define dataset paths
benign_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/benign/400X"
malignant_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/malignant/400X"

# Function to load image paths
def load_image_paths(dir_path):
    return [os.path.join(dir_path, img) for img in os.listdir(dir_path) if img.endswith('.png')]

benign_images = load_image_paths(benign_dir)
malignant_images = load_image_paths(malignant_dir)

print(f"Total Benign Images: {len(benign_images)}")
print(f"Total Malignant Images: {len(malignant_images)}")

# Create labels (0 = Benign, 1 = Malignant)
benign_labels = [0] * len(benign_images)
malignant_labels = [1] * len(malignant_images)

# Combine images and labels
all_images = np.array(benign_images + malignant_images)
all_labels = np.array(benign_labels + malignant_labels)

# Split into training (70%) and testing (30%)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.3, stratify=all_labels, random_state=42
)

print(f"Training samples: {len(train_images)}")
print(f"Testing samples: {len(test_images)}")

# Function to preprocess images
def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)  # Decode PNG images
    img = tf.image.resize(img, [224, 224])  # Resize to 224x224
    img = img / 255.0  # Normalize pixel values
    return img, label

# Create TensorFlow datasets
BATCH_SIZE = 128

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.map(process_path).shuffle(1000).batch(BATCH_SIZE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = test_dataset.map(process_path).batch(BATCH_SIZE)

# Ensure testing dataset is not empty
if sum(1 for _ in test_dataset) == 0:
    raise ValueError("Testing dataset is empty. Adjust your dataset split.")

# Load VGG16 without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classifier on top
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)  # Binary classification

# Create the model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
EPOCHS = 100
history = model.fit(train_dataset, epochs=EPOCHS)

# Evaluate model
test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()
from sklearn.metrics import confusion_matrix

# Evaluate model
test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()

# Get confusion matrix values
tn, fp, fn, tp = confusion_matrix(test_labels, test_preds).ravel()

# Calculate IBA
iba = (tp / (tp + fn)) + (tn / (tn + fp)) - 1

# Output results
f1 = f1_score(test_labels, test_preds)
gmean = geometric_mean_score(test_labels, test_preds)

loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"G-Mean: {gmean:.4f}")
print(f"Informedness (IBA): {iba:.4f}")

Total Benign Images: 588
Total Malignant Images: 1232
Training samples: 1274
Testing samples: 546
Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 180s 4s/step - accuracy: 0.6058 - loss: 1.9543
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 40s 663ms/step - accuracy: 0.7180 - loss: 0.7157
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 587ms/step - accuracy: 0.7347 - loss: 0.5236
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 21s 598ms/step - accuracy: 0.8036 - loss: 0.4434
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 586ms/step - accuracy: 0.8728 - loss: 0.3545
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 23s 684ms/step - accuracy: 0.8966 - loss: 0.3030
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 19s 587ms/step - accuracy: 0.9104 - loss: 0.2799
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 684ms/step - accuracy: 0.9231 - loss: 0.2413
Epoch 9/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 21s 592ms/step - accuracy: 0.9412 - loss: 0.2174
Epoch 10/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 609ms/step - accuracy: 0.9464 - loss: 0.2025
Epoch 11/100
10/10 ━━━━

# Here is a table summarizing the performance metrics for all magnification factors (40X, 100X, 200X, 400X) from notebook:

In [1]:
import pandas as pd

# Create a DataFrame with the provided values
data = {
    'Magnification': ['40X', '100X', '200X', '400X'],
    'Test Accuracy': [0.8531, 0.8754, 0.8692, 0.8462],
    'F1 Score': [0.8952, 0.9122, 0.9082, 0.8906],
    'G-Mean': [0.8105, 0.8301, 0.8197, 0.7939],
    'Informedness (IBA)': [0.6329, 0.6730, 0.6542, 0.6061]
}

df = pd.DataFrame(data)

# Display the table with formatting
styled_df = df.style \
    .format({
        'Test Accuracy': '{:.4f}',
        'F1 Score': '{:.4f}',
        'G-Mean': '{:.4f}',
        'Informedness (IBA)': '{:.4f}'
    }) \
    .set_properties(**{'text-align': 'center'}) \
    .set_table_styles([
        {'selector': 'th', 'props': [('background-color', '#000000'), ('font-weight', 'bold'), ('color', 'white')]}
    ]) \
    .hide(axis='index')

styled_df


Magnification,Test Accuracy,F1 Score,G-Mean,Informedness (IBA)
40X,0.8531,0.8952,0.8105,0.6329
100X,0.8754,0.9122,0.8301,0.6730
200X,0.8692,0.9082,0.8197,0.6542
400X,0.8462,0.8906,0.7939,0.6061
